In [8]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision.transforms.functional import to_tensor, to_pil_image
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

def enhance_image(input_image_path, output_image_path, scale=4, model_path='weights/RealESRGAN_x4plus.pth'):
    # Set device to CPU (or GPU if available)
    device = torch.device("cpu")

    # Load the pre-trained RRDBNet model
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=scale).to(device)

    # Initialize the Real-ESRGANer
    upsampler = RealESRGANer(scale=scale, model_path=model_path, model=model, tile=0, tile_pad=10, pre_pad=0, half=True)

    # Open the input image
    img = Image.open(input_image_path)

    # Convert the image to a NumPy array (instead of tensor)
    img_numpy = np.array(img)

    # Pass through the model (ensure the image is a NumPy array, not a tensor)
    with torch.no_grad():
        enhanced_img_numpy, _ = upsampler.enhance(img_numpy, outscale=scale)

    # Convert the enhanced NumPy array back to a PIL image
    enhanced_img = Image.fromarray(enhanced_img_numpy)

    # Save the enhanced image
    enhanced_img.save(output_image_path)
    print(f'Enhanced image saved at {output_image_path}')

# Provide your input and output image paths directly
input_image_path = 'im1.jpeg'  # Replace with your actual input image path
output_image_path = 'img.jpeg'

# Modify model path and scale if needed
model_path = 'ESRGAN/Real-ESRGAN/weights/RealESRGAN_x4plus.pth'
scale = 4

# Enhance the image
enhance_image(input_image_path, output_image_path, scale=scale, model_path=model_path)

Enhanced image saved at img.jpeg


In [ ]:
model_path = 'ESRGAN/Real-ESRGAN/weights/RealESRGAN_x4plus.pth'


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the BEL sitemap
url = 'https://bel-india.in/sitemap/'

# Send a GET request to the sitemap page
response = requests.get(url)

# Parse the content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Dictionary to store products and subproducts
defence_products = {}

# Find all <a> tags and filter for 'defence' links
links = soup.find_all('a')

# Loop through links and structure the data
for link in links:
    href = link.get('href')
    if href and 'defence' in href:  # Check if href exists
        product_name = link.text.strip()
        defence_products[product_name] = []
        # Now, find subproducts (if any) related to this product
        subproduct_section = link.find_next('ul')
        if subproduct_section:
            subproducts = subproduct_section.find_all('li')
            for subproduct in subproducts:
                defence_products[product_name].append(subproduct.text.strip())

# Convert dictionary to a DataFrame for better readability
df = pd.DataFrame([(key, ', '.join(value)) for key, value in defence_products.items()], columns=['Product', 'Subproducts'])

# Display the DataFrame
print(df)

                                              Product  \
0                                             Defence   
1                      Defence Communication Products   
2                                   Land-Based Radars   
3                                       Naval Systems   
4                          Electronic Warfare Systems   
5                                    AvionicsAvionics   
6                        Electro OpticsElectro Optics   
7   Tank & Armoured Fighting Vehicle Electronic Sy...   
8                        Weapon SystemsWeapon Systems   
9                              C4I SystemsC4I Systems   
10                                 Navigation Systems   
11                                           Upgrades   
12                   Shelters & MastsShelters & Masts   
13                               SimulatorsSimulators   
14                                          Batteries   
15           Components / DevicesComponents / Devices   
16                             

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import os

def scrape_images(product_name, folder_path, num_images=200):
    # Set up Selenium WebDriver for Safari
    driver = webdriver.Safari()  # Safari WebDriver is built-in for macOS
    
    driver.get("https://www.bing.com/images")
    
    # Enter the product name in the search box and press Enter
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(product_name)
    search_box.send_keys(Keys.RETURN)
    
    # Give time for the search results to load
    time.sleep(3)
    
    # Scroll down to load more images
    image_urls = set()
    scroll_pause_time = 3  # Adjust this time if necessary
    max_scroll_attempts = 10  # Adjust this value if needed
    scroll_attempts = 0

    while len(image_urls) < num_images and scroll_attempts < max_scroll_attempts:
        try:
            # Scroll down
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)  # Wait for images to load
            
            # Find image elements
            images = driver.find_elements(By.CSS_SELECTOR, "img.mimg")
            print(f"Found {len(images)} images")  # Debugging line
            
            if len(images) == 0:
                print("No images found. Check CSS selectors or page layout.")
                break

            # Extract image URLs
            for img in images:
                try:
                    src = img.get_attribute('src')
                    if src and 'http' in src:
                        image_urls.add(src)
                        if len(image_urls) >= num_images:
                            break
                except Exception as e:
                    print(f"Error while extracting URL: {e}")
                    continue
            
            # Increase the scroll attempts count
            scroll_attempts += 1
            print(f"Scroll attempt {scroll_attempts}")

        except Exception as e:
            print(f"Error during scrolling or loading images: {e}")
            break

    # Close the browser
    driver.quit()

    # Create a directory to save images if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Download images to the specified folder
    for idx, url in enumerate(image_urls):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                with open(f"{folder_path}/{product_name}_{idx+1}.jpg", 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded {product_name}_{idx+1}.jpg to {folder_path}")
            else:
                print(f"Failed to download {url}: Status code {response.status_code}")
        except Exception as e:
            print(f"Failed to download {url}: {e}")

if __name__ == "__main__":
    product_name = "Secure Digital Handheld Radio-5W BEL"  # You can change this to any product name
    folder_path = "images/secure_hand_held_raadio"  # Specify the folder path here (replace with the actual path)
    scrape_images(product_name, folder_path, num_images=200)

Found 70 images
Scroll attempt 1
Found 104 images
Scroll attempt 2
Found 138 images
Scroll attempt 3
Found 171 images
Scroll attempt 4
Found 204 images
Scroll attempt 5
Found 239 images
Scroll attempt 6
Downloaded Secure Digital Handheld Radio-5W BEL_1.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_2.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_3.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_4.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_5.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_6.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_7.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_8.jpg to images/secure_hand_held_raadio
Downloaded Secure Digital Handheld Radio-5W BEL_9.jpg to images/secure_hand_held_raadio
Downl